In [ ]:
from torchvision import datasets, transforms
import torch
import neat
# TODO: These are a beginning of an own implemenation of NEAT, not used anywhere yet.
class Genome:
    def __init__(self, key):
        self.key = key

class NEAT:
    def __init__(self, inputNodes, outputNodes):
        self.inputLayer =  torch.nn.Linear(in_features= inputNodes,out_features=1)
        self.relu1 = torch.nn.ReLU()
        self.outputLayer = torch.nn.Linear(1,outputNodes)
        self.model = [self.inputLayer, self.relu1,self.outputLayer]
        self.nLayers = 1
        

class Model:
    def __init__(self,inputNodes,outputNodes):
        self.inputNodes = inputNodes
        self.outputNodes = outputNodes


In [3]:
from torchvision import datasets, transforms
import torch

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

dataset = datasets.MNIST(root='../data', train=True, download=True, transform=transform);
trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True);


In [45]:
train_dataset = dataset.train_data
train_dataset = train_dataset.view(-1,784)[:20]
train_labels = dataset.train_labels
train_labels = train_labels[:20]

/home/nils/Documents/naturliga_beräkningsmetoder/project/NEAT-PSO/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:76: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/nils/Documents/naturliga_beräkningsmetoder/project/NEAT-PSO/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:66: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [44]:
train_dataset.view(-1,784)[:20].shape

torch.Size([20, 784])

In [50]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for xi, xo in zip(train_dataset, train_labels):
            output = torch.tensor(net.activate(xi))
            genome.fitness -= torch.nn.functional.cross_entropy(output , xo)

In [53]:
def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))
    print("before winner")
    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 2)
    print("after winner")
    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    for xi, xo in zip(train_dataset, train_labels):
        print("inside foor loop")
        xi = xi.flatten()
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

In [54]:
run('NEAT/config-feedforward')

before winner

 ****** Running generation 0 ****** 

Population's average fitness: -44.03394 stdev: 1.91448
Best fitness: -39.70070 - size: (10, 7840) - species 1 - id 117
Average adjusted fitness: 0.520


TypeError: type Tensor doesn't define __round__ method